# Milestone 3 PREP - CS109B Project Group 29
**"Predicting Movie Genres"**<br>
**CS109B (Spring 2017) Final Project Group 29**<br>
Calvin J Chiew, Tim Hagmann, Ji Hua<br>
TF: Taira Miyatake<br>

## Import Libraries

In [3]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import random

## Dummy-code Cast/Director/Writer

In [3]:
data = pd.read_csv("data/imdb_movie_info_100k_combined.txt", sep="\t")

company_dict={}
for i in range(data.shape[0]):
    com_line=str(data.iloc[i,4])
    words=com_line.split(",")
    for word in words:
        if company_dict.get(word,None)!=None:
            company_dict[word]+=1
        else:
            company_dict[word]=1

In [4]:
import operator
sorted_x = sorted(company_dict.items(), key=operator.itemgetter(1))
sorted_x.reverse()
count=0
for key in sorted_x:
    print key
    count+=1
    if count>=75:
        break

('nan', 9073)
('0000636', 128)
('0393250', 116)
('0939147', 107)
('0012148', 105)
('0001238', 103)
('0540789', 84)
('0000122', 79)
('0000175', 70)
('0002759', 70)
('0907778', 68)
('0002042', 64)
('0002192', 61)
('0880493', 59)
('0372942', 58)
('0140643', 56)
('0342278', 55)
('0044156', 52)
('0115669', 52)
('0151653', 51)
('0000419', 51)
('0176420', 51)
('0829044', 49)
('0776433', 49)
('0139541', 49)
('0241416', 49)
('0073053', 49)
('0000590', 49)
('0169540', 48)
('0000005', 48)
('0000095', 48)
('0800319', 47)
('0236279', 46)
('0000108', 45)
('0179278', 45)
('0144689', 45)
('0001348', 44)
('0408488', 44)
('0511807', 44)
('0024589', 43)
('0425913', 42)
('0700578', 42)
('0093501', 42)
('0020571', 42)
('0001175', 41)
('1270819', 41)
('0611794', 40)
('0332417', 40)
('0342303', 40)
('0359034', 40)
('0288111', 39)
('0002086', 39)
('0001031', 39)
('0317450', 39)
('0948634', 39)
('0026153', 39)
('0924901', 38)
('0073714', 38)
('0682481', 38)
('0203246', 37)
('0258493', 37)
('0316363', 37)
('022

---

## Load Dataframes

In [4]:
# tmdb dataset
tmdb = pd.read_table('data/tmdb_movie_info_100k.txt')

# convert release_date to release_year and release_month
tmdb['release_year'] = pd.to_datetime(tmdb['release_date'], errors='coerce').dt.year
tmdb['release_month'] = pd.to_datetime(tmdb['release_date'], errors='coerce').dt.month

tmdb.head()

C:\Users\Calvin\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (0,1,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,tmdb_id,imdb_id,title,plot,poster_path,release_date,budget,revenue,runtime,popularity,user_rating,production_company,genres,release_year,release_month
0,2,0094675,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,/gZCJZOn4l0Zj5hAxsMbxoS6CL0u.jpg,1988-10-21,0.0,0.0,69,0.623849,6.8,"Villealfa Filmproduction Oy,Finnish Film Found...","Drama,Crime",1988.0,10.0
1,3,0092149,Shadows in Paradise,"An episode in the life of Nikander, a garbage ...",/7ad4iku8cYBuB08g9yAU7tHJik5.jpg,1986-10-16,0.0,0.0,76,0.288572,6.8,Villealfa Filmproduction Oy,"Drama,Comedy",1986.0,10.0
2,5,0113101,Four Rooms,It's Ted the Bellhop's first night on the job....,/eQs5hh9rxrk1m4xHsIz1w11Ngqb.jpg,1995-12-25,4000000.0,4300000.0,98,1.768986,6.4,"Miramax Films,A Band Apart","Crime,Comedy",1995.0,12.0
3,6,0107286,Judgment Night,"While racing to a boxing match, Frank, Mike, J...",/lNXmgUrP6h1nD53gkFh4WDzT6RZ.jpg,1993-10-15,0.0,12136938.0,110,0.676269,6.6,"Largo Entertainment,JVC Entertainment Networks","Action,Thriller,Crime",1993.0,10.0
4,8,0825671,Life in Loops (A Megacities RMX),Timo Novotny labels his new project an experim...,/8YyIjOAxwzD3fZMdmJrfiApod4l.jpg,2006-01-01,42000.0,0.0,80,0.125477,5.2,inLoops,Documentary,2006.0,1.0


---

In [5]:
# rating info from IMDb
imdb = pd.read_table('data/imdb_movie_info_100k_combined.txt')
imdb = imdb[['tmdb_id', 'USA_rating']]
imdb.head()

,tmdb_id,USA_rating
0,2,Not Rated
1,3,NaN
2,5,R
3,6,R
4,8,NaN


In [6]:
imdb.USA_rating.unique()

array(['Not Rated', nan, 'R', 'PG', 'G', 'PG-13', 'Approved', 'TV-14', 'E',
       'Unrated', 'NC-17', 'TV-MA', 'Passed', 'TV-PG', 'M', 'TV-G', 'X',
       'GP', 'M/PG', 'TV-Y', 'Approved Certificate #17398)', 'TV-Y7', '12',
       '(Banned)', 'T', 'Open', 'AO', 'G '], dtype=object)

In [7]:
# clean up rating
imdb = imdb.replace('Not Rated', np.nan)
imdb = imdb.replace('Approved', np.nan)
imdb = imdb.replace('Unrated', np.nan)
imdb = imdb.replace('Passed', np.nan)
imdb = imdb.replace('Approved Certificate #17398)', np.nan)
imdb = imdb.replace('Open', np.nan)
imdb = imdb.replace('G ', 'G')

In [8]:
imdb.USA_rating.unique()

array([nan, 'R', 'PG', 'G', 'PG-13', 'TV-14', 'E', 'NC-17', 'TV-MA',
       'TV-PG', 'M', 'TV-G', 'X', 'GP', 'M/PG', 'TV-Y', 'TV-Y7', '12',
       '(Banned)', 'T', 'AO'], dtype=object)

---

In [9]:
# load genres
genres = pd.read_csv('data/genres_70k_new.txt')

# drop TVMovie as 0 observations
genres = genres.drop('TVMovie', 1)

# generate new Y variable -- number of genres
genres['num_genres'] = genres.ix[:, 1:19].sum(axis=1)

genres.head()

,tmdb_id,Adventure,Fantasy,Animation,Drama,Horror,Action,Comedy,History,Western,...,Crime,Documentary,Science Fiction,Mystery,Music,Romance,Family,War,Foreign,num_genres
0,2,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,2
1,3,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2
2,5,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,2
3,6,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,3
4,8,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1


---

In [10]:
# bag-of-words analysis for title and plot
raw = pd.read_table('data/tmdb_movie_info_100k.txt')

# concatenate title and plot together 
raw['title_plot'] = raw['title'].astype(str) + ' ' + raw['plot'].astype(str)

# drop all columns except title and plot
raw = raw[['tmdb_id', 'imdb_id', 'title_plot']]

# view first few rows
raw.head()

,tmdb_id,imdb_id,title_plot
0,2,0094675,Ariel Taisto Kasurinen is a Finnish coal miner...
1,3,0092149,Shadows in Paradise An episode in the life of ...
2,5,0113101,Four Rooms It's Ted the Bellhop's first night ...
3,6,0107286,"Judgment Night While racing to a boxing match,..."
4,8,0825671,Life in Loops (A Megacities RMX) Timo Novotny ...


In [11]:
# create bag-of-words matrix
from sklearn.feature_extraction.text import TfidfVectorizer
# ignore stop words and only consider words that make up at least 1% of the corpus
vectorizer = TfidfVectorizer(stop_words='english', min_df=0.01)
corpus = raw['title_plot'].values
X = vectorizer.fit_transform(corpus)
vocab = vectorizer.get_feature_names()
len(vocab)

389

In [12]:
bagofwords = pd.DataFrame(X.toarray(), columns=vocab)
bagofwords = bagofwords.add_prefix('word_')
bagofwords = pd.concat([raw[['tmdb_id']], bagofwords], axis=1)
bagofwords.head()

,tmdb_id,word_accident,word_action,word_adventure,word_affair,word_age,word_agent,word_america,word_american,word_amp,...,word_work,word_working,word_works,word_world,word_writer,word_written,word_year,word_years,word_york,word_young
0,2,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0
1,3,0.0,0.0,0.000000,0.665059,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0
2,5,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0
3,6,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.394097,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0
4,8,0.0,0.0,0.184482,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.119018,0.0,0.180166,0.0,0.0,0.165786,0.0


---

In [13]:
companies = pd.read_csv('data/companies_100k.txt')
companies.head()

,tmdb_id,Warner Bros,Metro-Goldwyn-Mayer MGM,Paramount Pictures,Universal Pictures,Twentieth Century Fox Film Corporation,Columbia Pictures Corporation,RKO Radio Pictures,Columbia Pictures,Canal+,...,United Artists,Mosfilm,Walt Disney Pictures,Shaw Brothers,New Line Cinema,Touchstone Pictures,Toho Company,TriStar Pictures,British Broadcasting Corporation BBC,Miramax Films
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Final Merge

In [14]:
# prepare tmdb dataframe
tmdb = tmdb.drop(['title', 'plot', 'poster_path', 'release_date', 'production_company', 'genres'], axis=1)
tmdb['tmdb_id'] = tmdb['tmdb_id'].apply(pd.to_numeric, errors='ignore')

In [25]:
# merge imdb info
merged1 = pd.merge(tmdb, imdb, how='left', on='tmdb_id')
merged1.head()

,tmdb_id,imdb_id,budget,revenue,runtime,popularity,user_rating,release_year,release_month,USA_rating
0,2,0094675,0.0,0.0,69,0.623849,6.8,1988.0,10.0,NaN
1,3,0092149,0.0,0.0,76,0.288572,6.8,1986.0,10.0,NaN
2,5,0113101,4000000.0,4300000.0,98,1.768986,6.4,1995.0,12.0,R
3,6,0107286,0.0,12136938.0,110,0.676269,6.6,1993.0,10.0,R
4,8,0825671,42000.0,0.0,80,0.125477,5.2,2006.0,1.0,NaN


In [26]:
merged1.shape

(76990, 10)

In [27]:
# merge production companies
companies['tmdb_id'] = companies['tmdb_id'].apply(pd.to_numeric, errors='ignore')
merged2 = pd.merge(merged1, companies, how='left', on='tmdb_id')
merged2.head()

,tmdb_id,imdb_id,budget,revenue,runtime,popularity,user_rating,release_year,release_month,USA_rating,...,United Artists,Mosfilm,Walt Disney Pictures,Shaw Brothers,New Line Cinema,Touchstone Pictures,Toho Company,TriStar Pictures,British Broadcasting Corporation BBC,Miramax Films
0,2,0094675,0.0,0.0,69,0.623849,6.8,1988.0,10.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3,0092149,0.0,0.0,76,0.288572,6.8,1986.0,10.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5,0113101,4000000.0,4300000.0,98,1.768986,6.4,1995.0,12.0,R,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,6,0107286,0.0,12136938.0,110,0.676269,6.6,1993.0,10.0,R,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8,0825671,42000.0,0.0,80,0.125477,5.2,2006.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
# merge bag-of-words
bagofwords['tmdb_id'] = bagofwords['tmdb_id'].apply(pd.to_numeric, errors='ignore')
merged3 = pd.merge(merged2, bagofwords, how='left', on='tmdb_id')
merged3.head()

,tmdb_id,imdb_id,budget,revenue,runtime,popularity,user_rating,release_year,release_month,USA_rating,...,word_work,word_working,word_works,word_world,word_writer,word_written,word_year,word_years,word_york,word_young
0,2,0094675,0.0,0.0,69,0.623849,6.8,1988.0,10.0,NaN,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0
1,3,0092149,0.0,0.0,76,0.288572,6.8,1986.0,10.0,NaN,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0
2,5,0113101,4000000.0,4300000.0,98,1.768986,6.4,1995.0,12.0,R,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0
3,6,0107286,0.0,12136938.0,110,0.676269,6.6,1993.0,10.0,R,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0
4,8,0825671,42000.0,0.0,80,0.125477,5.2,2006.0,1.0,NaN,...,0.0,0.0,0.0,0.119018,0.0,0.180166,0.0,0.0,0.165786,0.0


In [29]:
# merge outcome variables
genres['tmdb_id'] = genres['tmdb_id'].apply(pd.to_numeric, errors='ignore')
merged4 = pd.merge(merged3, genres, how='left', on='tmdb_id')
merged4.head()

,tmdb_id,imdb_id,budget,revenue,runtime,popularity,user_rating,release_year,release_month,USA_rating,...,Crime,Documentary,Science Fiction,Mystery,Music,Romance,Family,War,Foreign,num_genres
0,2,0094675,0.0,0.0,69,0.623849,6.8,1988.0,10.0,NaN,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1,3,0092149,0.0,0.0,76,0.288572,6.8,1986.0,10.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2,5,0113101,4000000.0,4300000.0,98,1.768986,6.4,1995.0,12.0,R,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,6,0107286,0.0,12136938.0,110,0.676269,6.6,1993.0,10.0,R,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
4,8,0825671,42000.0,0.0,80,0.125477,5.2,2006.0,1.0,NaN,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [30]:
merged4.shape

(80086, 439)

In [55]:
merged4.to_csv('data/merged.csv')

## Assess Missing Data

In [32]:
# generate table to assess extent of missing data
def missing_values_table(df): 
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum()/len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(columns = {0: 'Missing Values', 1: '% Missing Values'})
    return mis_val_table_ren_columns

missing_values_table(merged4)

,Missing Values,% Missing Values
tmdb_id,0,0.000000
imdb_id,0,0.000000
budget,3,0.003746
revenue,3,0.003746
runtime,4,0.004995
popularity,3,0.003746
user_rating,5,0.006243
release_year,270,0.337138
release_month,270,0.337138
USA_rating,58393,72.912869


In [33]:
# Handle missings
merged4.loc[:, merged4.dtypes == "float64"] = merged4.loc[:, merged4.dtypes == "float64"].fillna(0.0)
merged4.loc[:, merged4.dtypes == "object"] = merged4.loc[:, merged4.dtypes == "object"].fillna('Missing')

missing_values_table(merged4)

,Missing Values,% Missing Values
tmdb_id,0,0.0
imdb_id,0,0.0
budget,0,0.0
revenue,0,0.0
runtime,0,0.0
popularity,0,0.0
user_rating,0,0.0
release_year,0,0.0
release_month,0,0.0
USA_rating,0,0.0


In [34]:
merged4.to_csv('data/merged2.csv')